# Stochastic Frank-Wolfe

From Reddi et al.

# TODO LIST

### Most important
1) Come calcolare argmax?

3) Inserisco calcolo gamma
4) Metto come parametri "dati" del modello la gamma e la T specificate nel teorema di convergenza
5) E' da inserire un early stopping? Teoricamente nello pseudocodice non c'e, ma se e' per questo non c'e' nemmeno nell'FW normale.

6) Aggungere pseudocodice
7) Aggiungere sezione oracle?
8) Altre cose da aggiungere?



## Introduction

Frank-Wolfe methods (in convex case) have gained recent interest in machine learning and optimization communities due to their projection-free property and their ability to explout structured constraints. However, our understanding of these algorithms in the nonconvex setting is fairly limited. Given the vast importance of nonconvex models in machine learning, we propose a stochastic Frank-Wolfe algorithm.

## Preliminaries

**Optimization problem.** We study optimization problem of the form:

$$ \underset{x \in \Omega}{min} \  F(x) = \mathbb{E_z}[f(x,z)] $$

We assume:
-  $F$ differentiable, possibly nonconvex, and *L-smooth*, i.e., its gradient is Lipschitz continuous with constant L: $$ || \nabla F(x) - \nabla F(y) || \leq L||x-y||, \ \ \ \ \ \forall x,y \in \Omega$$ where $||.||$ denotes the $\ell^2$ norm.

-  $f$ differentiable, possibly nonconvex, and $G$-Lipschitz, i.e.: $$ || \nabla f(x,z) || \leq G, \ \ \ \ \ \forall x \in \Omega, z \in \Xi $$ where $||.||$ denotes the $\ell^2$ norm.

- $\Omega$ is convex and compact.

**Convergence criteria.** As convergence criteria, we use the following quantity, typically referred to as *Frank-Wolfe gap*:

$$G(x) = \underset{v \in \Omega}{max} \langle v-x, -\nabla F(x) \rangle$$

In nonconvex functions, G(x) = 0 if and only if x is a stationary point. To state our converge results, we will also need the following bound:

$$\beta \geq \frac{2(F(x_0)-F(x*))}{LD^{2}}$$

given some (unspecified) initial point $x_0 \in \Omega$.

**Oracle model.** Has to be added? It is used to compare speed of different algorithms, but we implement only one algorithm.

## SFW algorithm

Follows the pseudocode of the SFW algorithm.

**TO COMPLETE**

- Samples {z_i} has been chosen independentely according to the distribution *P*. Thus, we obtain an unbiased estimate of the gradient.
- The step sizes $\{\gamma_i\}^{T-1}_{i=0}$ and the minibatch sizes $\{b_t\}$ are the key parameters of the model, and must be chosen appropriately in order to ensure convergence of the algorithm. SEE THE CONVERGENCE RESULT
- Note that the output is randomly selected from all the iterates of the algorithm.

The following key result applies for nonconvex SFW.

**Theorem X.** *Consider the presented SFW algorithm and its described stochastic setting. Then, the output* $x_a$ *with parameters* $\gamma_t = \gamma = \sqrt{\frac{2(F(x_0)-F(x^{*}))}{TLD^2 \beta}}, b_t = b = T$ *for all* t \in {0,...,T-1}, *satisfies the following bound:*

$$ \mathbb{E}[G(x_a)] \leq \frac{D}{\sqrt{T}} \left( G + \sqrt{\frac{2L(F(x_0)-F(x^{*}))}{\beta}}(1+\beta)\right)$$

*where* x^{*} *is an optimal solution to the stochastic problem described*.

An immediate consequence of Theorem X is the following complexity result for SFW.

**Corollary X.**. *Under the setting of Theorem X, the SFO complexity and LO complexity of SFW algorithm are* O(1/ $\varepsilon^{4}$ ) *and* O(1/$\varepsilon^{2}$) *respectively*.

Note the SFO and LO complexity of nonconvex SFW is slightly worse than complexity of SFW for convex problems. Note also that we used a fixed step size and minibatch size. One can derive an essentially similar result using a decreasing step size and an increasing minibatch size.



# CODE

In [74]:
# Imports
import numpy as np
import time
import sklearn

from sklearn.linear_model import Lasso
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## LINEAR REGRESSION WITH CONSTRAINT

I implemented the linear regression at the end of Combettes

## -------

In [ ]:
def linear_SFW_gradient(xk, sample, y_batch):
    grad = (np.shape(sample)[0])^(-1) * (y_batch - np.matmul(sample,xk))^2
    return grad

def linear_SFW_argmax(grad):
    # What is the element in the constrained set which maximise the multiplication between itself and -gradient?
    



def linear_SFW_predict(xt, X_test):
    return np.matmul(X_test,xt)

Implementation of the proper SFW algorithm.

Notation conversion from pseudo-code to code:
- $x_0$ stays the same;
- *T* (number of iterations) becomes *epochs*;
- *gamma_i* stays the same (you give an input a specified rule and, depeding on it, gamma's value is computed)
- *b_i* becomes *batch_dim*.

It also add the following parameters:
- *sample*, *y_true*: the sample covariates and outputs, respectively;
- *task*: depending on the input, it performs a different task.
- *check*: ....

In [76]:
def linear_SFW(samp, y_true, x0, batch_dim, check, gamma = "theorem", epochs=101):

    # 1. Choose the starting point
    xt = []
    xt.append(x0)

    # this section add a column of
    # 1 in front of the sample (to compute the gradient of beta0)
    samp = np.concatenate((np.ones((samp.shape[0], 1)), samp), axis=1)

    acc_list = []
    time_list = []

    start_time = time.process_time()

    for t in range(epochs):

        # Uniformly randomly pick i.i.d. samples {z1, ..., z}
        indices = np.random.choice(samp.shape[0], batch_dim, replace=False)
        y_batch = y_true[indices]
        samp_batch = samp[indices]

        # 3. Compute the search direction
        grad = linear_SFW_gradient(xt[t], samp_batch, y_batch)
        vt = linear_SFW_argmax(-grad)

        # NA. Compute the step size
        if gamma == "diminishing":
            gam = 1/(t+1)
        # elif gamma == "theorem":
            # gam = TO COMPLETE

        # Compute the update direction and update the iterate:
        dt = vt - xt[t]
        xt.append(xt[t] + gam*dt)

        # Store accuracy and time
        if(t % check == 0):
            mse = mean_squared_error(y_true, np.matmul(samp, xt[t]))
            acc_list.append(mse)
            time_list.append(time.process_time() - start_time)

            print(f"The MSE at iteration {t} is: {mse}")           

    index = np.random.choice(range(epochs))
    x_fin = xt[index]
    return x_fin, np.array(acc_list), np.array(time_list)


This section use SFW on a linear regression task and compare it to the result obtained through sklearn.

In [79]:
batch_dim = 15
lamb = 0.1
gamma = "diminishing"
epochs = 100
check = 10

diabetes = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=42)
x0 = np.zeros(X_train.shape[1]+1)

# Compute Lasso regression both with SFW and scikit
SFW_weights, acc, tempo = linear_SFW(X_train, y_train, x0, batch_dim, check, gamma, epochs)
SFW_pred = np.matmul(np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1), SFW_weights)
SFW_mse = mean_squared_error(y_test, SFW_pred)

sklearn_pred = Lasso(alpha = lamb).fit(X_train,y_train).predict(X_test)
sklearn_mse = mean_squared_error(y_test, sklearn_pred)

print(f"The MSE predicted by Sklearn is: {sklearn_mse}\nThe MSE predicted by SFW is: {SFW_mse}")



The MSE at iteration 0 is: 29711.32294617564
The MSE at iteration 10 is: 29680.585637393768
The MSE at iteration 20 is: 29680.585637393768
The MSE at iteration 30 is: 29680.585637393768
The MSE at iteration 40 is: 29680.585637393768
The MSE at iteration 50 is: 29680.585637393768
The MSE at iteration 60 is: 29680.585637393768
The MSE at iteration 70 is: 29680.585637393768
The MSE at iteration 80 is: 29680.585637393768
The MSE at iteration 90 is: 29680.585637393768
The MSE predicted by Sklearn is: 2798.193485169719
The MSE predicted by SFW is: 26519.439213483143


# MENATE SUL LASSO - Per ora non me la sento ancora di buttarle :)

The following section implements gradient and argmax computation in case of SFW is used for LASSO regression. We chose LASSO regression as it can be implemented as a linear program.

LASSO's loss function:

$$L(\beta) = \frac{1}{2n} \cdot (y-X\beta)^{2} + \lambda \cdot ||\beta||_{1}$$

Thus, the j-th partial derivative of the loss is:

$$ \frac{\partial L(\beta)}{\partial \beta_j} =  \frac{1}{n} \cdot (y-X\beta)^{2} \cdot x_{j} + sign(\lambda)$$

As we said, LASSO can be implemented as a linear program. If we choose this way, the optimization problem becomes:

$$ \textup{minimize} \ \ \  \frac{1}{2n} \cdot (\y-X\beta)^2$$
$$ \textup{subject to} \ \ \  \beta \geq 0 \textup{and} X^{T} \cdot (y-X\beta)$$
Where t is a scalar parameter that controls the sparsity of the solution. This is basically the Ordinary Least Squares problem subject to the constraint.

The j-th partial derivative of the loss becomes:

$$ -2 \cdot ||y - X\beta|| \cdot x_j $$

Link to the guy which implemented "argmin" original function:
https://github.com/paulmelki/Frank-Wolfe-Algorithm-Python/blob/master/frank_wolfe.py

In [ ]:
def lasso_SFW_gradient(xk, sample, y_batch):
    # STANDARD IMPLEMENTATION
    # grad = (1/m) * X^T * (X * xk - y_true) + lambda * sign(xk)
    #grad = (1/sample.shape[0]) * np.matmul(sample.T, (np.matmul(sample,xk) - y_batch)) \
    #    + lamb*np.sign(xk)

    # LINEAR PROGRAMMING IMPLEMENTATION
    grad = -2 * 
    return grad

def lasso_SFW_argmax(grad):

    # ???

    # Initialize the zero vector
    xk_hat = np.zeros(len(grad))

    # Check if all coordinates of x are 0
    # If they are, then the Oracle contains zero vector
    if (grad != 0).sum() == 0:
        return xk_hat

    # Otherwise, follow the following steps
    else:
        # Compute the (element-wise) absolute value of x
        a = abs(grad)
        # Find the first coordinate that has the maximum absolute value
        i = np.nonzero(a == max(a))[0][0]
        # Compute s
        xk_hat[i] = - np.sign(grad[i]) * lamb
        return xk_hat

def lasso_SFW_predict(xt, X_test):
    return np.matmul(test_1,xt)